In [1]:
import pandas as pd
import numpy as np
import os
import re
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
import seaborn as sns
import copy
from sklearn import preprocessing
import pickle

In [2]:
def compute_ci_95(arr):
    """
    Helper function to calculate 95% interval given an array
    """
    return [np.percentile(arr, 2.5), np.percentile(arr, 97.5)]

# Performing Causal Analysis (Mortality after 28 days)

In [548]:
# Retrieving merged_data (look at tte_paper_experimentations_orig.ipynb on how to generate)
full_data_df = pd.read_csv("full_data_df_no_index.csv")
### ONLY RUN FOR "merged_final_feng_predictions_from_non_feng_annotated.csv"
feng_final_predictions = pd.read_csv("...")

### Only i2c2
# pred_mimic_df = pd.read_csv("...")

### Mixed i2c2 and annotated mimic
# pred_mimic_df = pd.read_csv("...")

In [ ]:
full_data_df.head()

In [ ]:
pred_mimic_df = pred_mimic_df.rename(columns={'SUBJECT_ID': 'subject_id'})
pred_mimic_df.head()

In [ ]:
pred_mimic_df.columns

In [551]:
merged_df = pd.merge(full_data_df, feng_final_predictions[["subject_id","SMOKING_STATUS"]], on=["subject_id"])
# merged_df = pd.merge(full_data_df, pred_mimic_df[["subject_id","SMOKING_STATUS"]], on=["subject_id"])

In [552]:
merged_df["SMOKING_STATUS"].value_counts()

SMOKING_STATUS
3    2617
1    1611
2     507
4      43
0      21
Name: count, dtype: int64

In [553]:
# Droppping 0 labels -- NO need to know
merged_df = merged_df.drop(merged_df[merged_df["SMOKING_STATUS"] == 0].index)
merged_df["SMOKING_STATUS"].value_counts()


SMOKING_STATUS
3    2617
1    1611
2     507
4      43
Name: count, dtype: int64

In [555]:
# Converting integers to weekdays
def int_to_weekday(row):
    r = int(row)
    if r == 0:
        return 'sunday'
    elif r == 1:
        return "monday"
    elif r == 2:
        return "tuesday"
    elif r == 3:
        return "wednesday"
    elif r == 4:
        return "thursday"
    elif r== 5:
        return "friday"
    else:
        return "saturday"

merged_df["icu_adm_weekday"] = merged_df["icu_adm_weekday"].apply(int_to_weekday)

In [556]:
merged_df["first_careunit"] = merged_df["first_careunit"].astype('category')
merged_df["first_careunit"] = merged_df["first_careunit"].cat.reorder_categories(["SICU", "MICU"])

merged_df["gender"] = merged_df["gender"].astype("category")
merged_df["gender"] = merged_df["gender"].cat.reorder_categories(["M", "F"])

merged_df["icu_adm_weekday"] = merged_df["icu_adm_weekday"].astype("category")

In [557]:
# Understanding Matrix of Error Adjustments
# confusion = [
#                 [8, 0, 2, 1],
#                 [4, 4, 3, 0],
#                 [1, 0, 14, 1],
#                 [1, 0, 1, 61]
#             ] # rows represent the ground truth labels and cols represents the predicted labels

# error_mat = [
#                 [8/11, 0, 2/11, 1/11],
#                 [4/11, 4/11, 3/11, 0],
#                 [1/16, 0, 14/16, 1/16],
#                 [1/63, 0, 1/63, 61/63]
#             ] # rows represent U* and cols represent U
confusion = [
                [167, 5, 17, 0],
                [20, 70, 22, 1],
                [13, 10, 330, 3],
                [8, 0, 12, 11]
            ] # rows represent the ground truth labels and cols represents the predicted labels

error_mat = [
                [167/189, 5/189, 17/189, 0/189],
                [20/113, 70/113, 22/113, 1/113],
                [13/356, 10/356, 330/356, 3/356],
                [8/31, 0, 12/31, 11/31]
            ] # rows represent U* and cols represent U
inverse = np.linalg.pinv(error_mat)
inverse

array([[ 1.14381104, -0.04416817, -0.10319517,  0.00355229],
       [-0.30640811,  1.64101196, -0.3008217 , -0.03378215],
       [-0.02849433, -0.0487633 ,  1.10221776, -0.02496012],
       [-0.80077785,  0.08531863, -1.12736834,  2.84282756]])

In [560]:
def generate_models(dataframe):
    '''
    Given a pre-processed MIMIC + proxy prediction dataframe, train three logistic regression models 
    using smf.logit. 
    The formula strings will be hard-coded into the function. The assumptions for these models are:
        1) Categorical smoking categories 
        2) Not all feature are binary, but at least the output (mort_28_day) and treatment (echo) 
           should be binary
    '''
    
    # Calculating P(y | u*, a, c) --> y ~ u* + a + c for each u* label in [1,2,3,4] 
    fstring = 'mort_28_day ~ echo + age + weight + saps + sofa + elix_score + vent + \
            vaso + icu_adm_hour + icd_chf + icd_afib + icd_renal + icd_liver + icd_copd + \
            icd_cad + icd_stroke + icd_malignancy + vs_heart_rate_first + vs_map_first + vs_temp_first + \
            lab_hemoglobin_first + lab_platelet_first + lab_wbc_first + lab_chloride_first + \
            lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_creatinine_first + \
            lab_potassium_first + vs_cvp_flag + lab_creatinine_kinase_flag + lab_bnp_flag + gender + \
            lab_troponin_flag + first_careunit + icu_adm_weekday + lab_ph_first + lab_pco2_first + \
            lab_po2_first + lab_lactate_first + sedative + C(SMOKING_STATUS)'
    eq1 = smf.logit(fstring, data=dataframe)
    eq1_model = eq1.fit(disp=0)
    
    # Calculating P(u* | a, c)
    f_string2 = "SMOKING_STATUS ~ echo + first_careunit + age + gender + weight + saps + sofa + elix_score + \
            vent + vaso + icu_adm_weekday + icu_adm_hour + icd_chf + icd_afib + icd_renal + icd_liver + \
            icd_copd + icd_cad + icd_stroke + icd_malignancy + vs_heart_rate_first + vs_map_first + \
            vs_temp_first + lab_hemoglobin_first + lab_platelet_first + lab_wbc_first + lab_ph_first + \
            lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_pco2_first + lab_creatinine_first + \
            lab_chloride_first + lab_potassium_first + lab_po2_first + lab_lactate_first + sedative + \
            vs_cvp_flag + lab_creatinine_kinase_flag + lab_bnp_flag" 
    eq2 = smf.mnlogit(f_string2, data=dataframe)
    eq2_model = eq2.fit(disp=0)
    
    # Calculating P(a|c)
    f_string3 = "echo ~ first_careunit + age + gender + weight + saps + sofa + elix_score + vent + \
                vaso + icu_adm_weekday + icu_adm_hour + icd_chf + icd_afib + icd_renal + icd_liver + icd_copd + \
                icd_cad + icd_stroke + icd_malignancy + vs_heart_rate_first + vs_map_first + vs_temp_first + \
                lab_hemoglobin_first + lab_platelet_first + lab_wbc_first + lab_ph_first + lab_chloride_first + \
                lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_pco2_first + lab_creatinine_first + \
                lab_potassium_first + lab_po2_first + lab_lactate_first + sedative + vs_cvp_flag + \
                lab_creatinine_kinase_flag + lab_bnp_flag + lab_troponin_flag"
    eq3 = smf.logit(f_string3, data=dataframe)
    eq3_model = eq3.fit(disp=0)
    
    return eq1_model, eq2_model, eq3_model

In [561]:
def risk_ratio(dataframe, model1, model2, model3):
    '''
    Given a pre-procesesed MIMIC + smoking proxy prediction dataframe as well as three trained models 
    from generate_models(), calculate the risk ratio as defined by: 
    causal_effect = summation(c,u){ p(c,u) * ( E[Y=1 | A=1,c,u*] / E[Y=1 | A=0,c,u*] ) }
    The assumptions of this function are:
        1) Smoking proxy predictions are categorical
        2) Treatment variable values are binary --> either 1 for receiving treatment or 0 for not 
           receiving treatment
        3) Order for model inputs matter:
            a) model1 = P(y | u*, a, c) --> y ~ u* + a + c
            b) model2 = P(u* | a, c)
            c) model3 = P(a | c)
        4) Default prediction is probability of getting 1 due to how statsmodels works
    '''
    
    tmp_df = None
    unique_smoking = [1,2,3,4]
    unique_echo = [1,0]
    exp_array = []
#     confusion = [
#                 [8, 0, 2, 1],
#                 [4, 4, 3, 0],
#                 [1, 0, 14, 1],
#                 [1, 0, 1, 61]
#             ] # rows represent the ground truth labels and cols represents the predicted labels

#     error_mat = [
#                     [8/11, 0, 2/11, 1/11],
#                     [4/11, 4/11, 3/11, 0],
#                     [1/16, 0, 14/16, 1/16],
#                     [1/63, 0, 1/63, 61/63]
#             ]
    confusion = [
                [167, 5, 17, 0],
                [20, 70, 22, 1],
                [13, 10, 330, 3],
                [8, 0, 12, 11]
            ] # rows represent the ground truth labels and cols represents the predicted labels

    error_mat = [
                    [167/189, 5/189, 17/189, 0/189],
                    [20/113, 70/113, 22/113, 1/113],
                    [13/356, 10/356, 330/356, 3/356],
                    [8/31, 0, 12/31, 11/31]
                ] # rows represent U* and cols represent U
    inverse = np.linalg.pinv(error_mat)
    # Getting P(A, c, y=1, u*) 
    prob_a1_c_y1_u = []
    prob_a0_c_y1_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to be a cateogrical value in [1,2,3,4]
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y1_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y1_u.append(output)
    
    # Getting P(A, c, y=0, u*)
    prob_a1_c_y0_u = []
    prob_a0_c_y0_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to be a cateogrical value in [1,2,3,4]
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = 1 - model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y0_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y0_u.append(output)
        
    
    # Getting P(Y=1 | A=1, C, U=0)
    num_0a = prob_a1_c_y1_u[0] * inverse[0][0] + prob_a1_c_y1_u[1] * inverse[1][0] + prob_a1_c_y1_u[2] * \
             inverse[2][0] + prob_a1_c_y1_u[3] * inverse[3][0]
    tmp_0a = prob_a1_c_y0_u[0] * inverse[0][0] + prob_a1_c_y0_u[1] * inverse[1][0] + prob_a1_c_y0_u[2] * \
             inverse[2][0] + prob_a1_c_y0_u[3] * inverse[3][0]
    denom_0a = num_0a + tmp_0a
    upper_0a = num_0a / denom_0a
    
    # Getting P(Y=1 | A=0, C, U=0)
    num_0b = prob_a0_c_y1_u[0] * inverse[0][0] + prob_a0_c_y1_u[1] * inverse[1][0] + prob_a0_c_y1_u[2] * \
             inverse[2][0] + prob_a0_c_y1_u[3] * inverse[3][0]
    tmp_0b = prob_a0_c_y0_u[0] * inverse[0][0] + prob_a0_c_y0_u[1] * inverse[1][0] + prob_a0_c_y0_u[2] * \
             inverse[2][0] + prob_a0_c_y0_u[3] * inverse[3][0]
    denom_0b = num_0b + tmp_0b
    lower_0b = num_0b / denom_0b
    
    comp_0 = upper_0a / lower_0b
    
    # Getting P(Y=1 | A=1, C, U=1)
    num_1a = prob_a1_c_y1_u[0] * inverse[0][1] + prob_a1_c_y1_u[1] * inverse[1][1] + prob_a1_c_y1_u[2] * \
             inverse[2][1] + prob_a1_c_y1_u[3] * inverse[3][1]
    tmp_1a = prob_a1_c_y0_u[0] * inverse[0][1] + prob_a1_c_y0_u[1] * inverse[1][1] + prob_a1_c_y0_u[2] * \
             inverse[2][1] + prob_a1_c_y0_u[3] * inverse[3][1]
    denom_1a = num_1a + tmp_1a
    upper_1a = num_1a / denom_1a
    
    # Getting P(Y=1 | A=0, C, U=1)
    num_1b = prob_a0_c_y1_u[0] * inverse[0][1] + prob_a0_c_y1_u[1] * inverse[1][1] + prob_a0_c_y1_u[2] * \
             inverse[2][1] + prob_a0_c_y1_u[3] * inverse[3][1]
    tmp_1b = prob_a0_c_y0_u[0] * inverse[0][1] + prob_a0_c_y0_u[1] * inverse[1][1] + prob_a0_c_y0_u[2] * \
             inverse[2][1] + prob_a0_c_y0_u[3] * inverse[3][1]
    denom_1b = num_1b + tmp_1b
    lower_1b = num_1b / denom_1b
    
    comp_1 = upper_1a / lower_1b
    
    # Getting P(Y=1 | A=1, C, U=2)
    num_2a = prob_a1_c_y1_u[0] * inverse[0][2] + prob_a1_c_y1_u[1] * inverse[1][2] + prob_a1_c_y1_u[2] * \
             inverse[2][2] + prob_a1_c_y1_u[3] * inverse[3][2]
    tmp_2a = prob_a1_c_y0_u[0] * inverse[0][2] + prob_a1_c_y0_u[1] * inverse[1][2] + prob_a1_c_y0_u[2] * \
             inverse[2][2] + prob_a1_c_y0_u[3] * inverse[3][2]
    denom_2a = num_2a + tmp_2a
    upper_2a = num_2a / denom_2a
    
    # Getting P(Y=1 | A=0, C, U=2)
    num_2b = prob_a0_c_y1_u[0] * inverse[0][2] + prob_a0_c_y1_u[1] * inverse[1][2] + prob_a0_c_y1_u[2] * \
             inverse[2][2] + prob_a0_c_y1_u[3] * inverse[3][2]
    tmp_2b = prob_a0_c_y0_u[0] * inverse[0][2] + prob_a0_c_y0_u[1] * inverse[1][2] + prob_a0_c_y0_u[2] * \
             inverse[2][2] + prob_a0_c_y0_u[3] * inverse[3][2]
    denom_2b = num_2b + tmp_2b
    lower_2b = num_2b / denom_2b
    
    comp_2 = upper_2a / lower_2b
    
    # Getting P(Y=1 | A=1, C, U=3)
    num_3a = prob_a1_c_y1_u[0] * inverse[0][3] + prob_a1_c_y1_u[1] * inverse[1][3] + prob_a1_c_y1_u[2] * \
             inverse[2][3] + prob_a1_c_y1_u[3] * inverse[3][3]
    tmp_3a = prob_a1_c_y0_u[0] * inverse[0][3] + prob_a1_c_y0_u[1] * inverse[1][3] + prob_a1_c_y0_u[2] * \
             inverse[2][3] + prob_a1_c_y0_u[3] * inverse[3][3]
    denom_3a = num_3a + tmp_3a
    upper_3a = num_3a / denom_3a
    
    # Getting P(Y=1 | A=0, C, U=3)
    num_3b = prob_a0_c_y1_u[0] * inverse[0][3] + prob_a0_c_y1_u[1] * inverse[1][3] + prob_a0_c_y1_u[2] * \
             inverse[2][3] + prob_a0_c_y1_u[3] * inverse[3][3]
    tmp_3b = prob_a0_c_y0_u[0] * inverse[0][3] + prob_a0_c_y0_u[1] * inverse[1][3] + prob_a0_c_y0_u[2] * \
             inverse[2][3] + prob_a0_c_y0_u[3] * inverse[3][3]
    denom_3b = num_3b + tmp_3b
    lower_3b = num_3b / denom_3b
    
    comp_3 = upper_3a / lower_3b
    
    # Getting P(u | c) 
    prob_u0_c = num_0a + tmp_0a + num_0b + tmp_0b
    prob_u1_c = num_1a + tmp_1a + num_1b + tmp_1b
    prob_u2_c = num_2a + tmp_2a + num_2b + tmp_2b
    prob_u3_c = num_3a + tmp_3a + num_3b + tmp_3b
    
    
    # Calculating total RR (updated version)
    total_upper = np.mean(upper_0a * prob_u0_c + upper_1a * prob_u1_c + upper_2a * prob_u2_c + \
                          upper_3a * prob_u3_c)
    total_lower = np.mean(lower_0b * prob_u0_c + lower_1b * prob_u1_c + lower_2b * prob_u2_c + \
                          lower_3b * prob_u3_c)
    rr = total_upper / total_lower

    sub_array = [np.mean(upper_0a * prob_u0_c) / np.mean(lower_0b * prob_u0_c),
                 np.mean(upper_1a * prob_u1_c) / np.mean(lower_1b * prob_u1_c),
                 np.mean(upper_2a * prob_u2_c) / np.mean(lower_2b * prob_u2_c),
                 np.mean(upper_3a * prob_u3_c) / np.mean(lower_3b * prob_u3_c)]
    return rr, sub_array

In [ ]:
m1, m2, m3 = generate_models(merged_df)
risk_ratio(merged_df, m1, m2, m3)

### Bootstrapping Merged Dataframe


In [545]:
def bootstrap_merged_data_rr(dataframe, model1, model2, model3):
    '''
    Given a pre-procesesed MIMIC + smoking proxy prediction dataframe as well as three trained models 
    from generate_models(), perform bootstrapping by shuffling the merged dataframe for
    risk ratio calculations. Iterations set to 100.
    The assumptions of this function are:
        1) Smoking proxy predictions are categorical
        2) Treatment variable values are binary --> either 1 for receiving treatment or 0 
           for not receiving treatment
        3) Order for model inputs matter:
            a) model1 = P(y | u*, a, c) --> y ~ u* + a + c
            b) model2 = P(u* | a, c)
            c) model3 = P(a | c)
    '''
    
    
    iterations = 100
    output = []
    sub_matrix = np.zeros((iterations, 4))
    for i in range(iterations):
        bt_df = dataframe.sample(frac=1, replace=True, ignore_index=True)
        rr, sub_array = risk_ratio(bt_df, model1, model2, model3)
        output.append(rr)

        for idx, c in enumerate(sub_array):
            sub_matrix[i, idx] = c
        sub_avg = sub_matrix.mean(axis=0)
        
    res_dict = {"bs_rr": sum(output) / len(output), "bs_arr_rr": output, "sub_avg_rr": sub_avg, \
                "sub_arr_rr": sub_matrix}
    
    return res_dict

bt_merged_rr = bootstrap_merged_data_rr(merged_df, m1, m2, m3)
bt_merged_rr["bs_rr"]

0.9225890002497992

In [433]:
# Computing 95% interval for risk ratio while boostrapping merged dataframe 
compute_ci_95(bt_merged_rr["bs_arr_rr"])

[0.9160855790478521, 0.9251527518267673]

In [434]:
# Computing 95% interval for risk ratio subgroups while bootstrapping merged dataframe
np.apply_along_axis(compute_ci_95, axis=0, arr=bt_merged_rr["sub_arr_rr"])

array([[ 0.905714  ,  0.91767654,  0.9176528 , -2.83883945],
       [ 0.92362036,  0.93654976,  0.92791562,  2.30819128]])

### Bootstrapping Error Rate Matrix

In [490]:
def risk_ratio_bootstrap(dataframe, model1, model2, model3, error_mat):
    '''
    Given a pre-procesesed MIMIC + smoking proxy prediction dataframe, three trained models 
    from generate_models(), and an error rate matrix, perform bootstrapping on error-rate matrix for
    risk-ratio calculations. Helper function for bootstrap()
    The assumptions of this function are:
        1) Smoking proxy predictions are categorical
        2) Treatment variable values are binary --> either 1 for receiving treatment or 0 
           for not receiving treatment
        3) Order for model inputs matter:
            a) model1 = P(y | u*, a, c) --> y ~ u* + a + c
            b) model2 = P(u* | a, c)
            c) model3 = P(a | c)
    '''
    
    tmp_df = None
    unique_smoking = [1,2,3,4]
    unique_echo = [1,0]
    exp_array = []
    
    # Inversing Error Rate Matrices
    inverse = np.linalg.pinv(error_mat)
    # print(inverse)
    
    # Getting P(A, c, y=1, u*) 
    prob_a1_c_y1_u = []
    prob_a0_c_y1_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to be a cateogrical value in [1,2,3,4]
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y1_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y1_u.append(output)
    
    # Getting P(A, c, y=0, u*)
    prob_a1_c_y0_u = []
    prob_a0_c_y0_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to be a cateogrical value in [1,2,3,4]
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = 1 - model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y0_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y0_u.append(output)
        
    
    # Getting P(Y=1 | A=1, C, U=0)
    num_0a = prob_a1_c_y1_u[0] * inverse[0][0] + prob_a1_c_y1_u[1] * inverse[1][0] + prob_a1_c_y1_u[2] * \
             inverse[2][0] + prob_a1_c_y1_u[3] * inverse[3][0]
    tmp_0a = prob_a1_c_y0_u[0] * inverse[0][0] + prob_a1_c_y0_u[1] * inverse[1][0] + prob_a1_c_y0_u[2] * \
             inverse[2][0] + prob_a1_c_y0_u[3] * inverse[3][0]
    denom_0a = num_0a + tmp_0a
    upper_0a = num_0a / denom_0a
    
    # Getting P(Y=1 | A=0, C, U=0)
    num_0b = prob_a0_c_y1_u[0] * inverse[0][0] + prob_a0_c_y1_u[1] * inverse[1][0] + prob_a0_c_y1_u[2] * \
             inverse[2][0] + prob_a0_c_y1_u[3] * inverse[3][0]
    tmp_0b = prob_a0_c_y0_u[0] * inverse[0][0] + prob_a0_c_y0_u[1] * inverse[1][0] + prob_a0_c_y0_u[2] * \
             inverse[2][0] + prob_a0_c_y0_u[3] * inverse[3][0]
    denom_0b = num_0b + tmp_0b
    lower_0b = num_0b / denom_0b
    
    # comp_0 = upper_0a / lower_0b
    
    
    # Getting P(Y=1 | A=1, C, U=1)
    num_1a = prob_a1_c_y1_u[0] * inverse[0][1] + prob_a1_c_y1_u[1] * inverse[1][1] + prob_a1_c_y1_u[2] * \
             inverse[2][1] + prob_a1_c_y1_u[3] * inverse[3][1]
    tmp_1a = prob_a1_c_y0_u[0] * inverse[0][1] + prob_a1_c_y0_u[1] * inverse[1][1] + prob_a1_c_y0_u[2] * \
             inverse[2][1] + prob_a1_c_y0_u[3] * inverse[3][1]
    denom_1a = num_1a + tmp_1a
    upper_1a = num_1a / denom_1a
    
    # Getting P(Y=1 | A=0, C, U=1)
    num_1b = prob_a0_c_y1_u[0] * inverse[0][1] + prob_a0_c_y1_u[1] * inverse[1][1] + prob_a0_c_y1_u[2] * \
             inverse[2][1] + prob_a0_c_y1_u[3] * inverse[3][1]
    tmp_1b = prob_a0_c_y0_u[0] * inverse[0][1] + prob_a0_c_y0_u[1] * inverse[1][1] + prob_a0_c_y0_u[2] * \
             inverse[2][1] + prob_a0_c_y0_u[3] * inverse[3][1]
    denom_1b = num_1b + tmp_1b
    lower_1b = num_1b / denom_1b
    
    # comp_1 = upper_1a / lower_1b
    
    # Getting P(Y=1 | A=1, C, U=2)
    num_2a = prob_a1_c_y1_u[0] * inverse[0][2] + prob_a1_c_y1_u[1] * inverse[1][2] + prob_a1_c_y1_u[2] * \
             inverse[2][2] + prob_a1_c_y1_u[3] * inverse[3][2]
    tmp_2a = prob_a1_c_y0_u[0] * inverse[0][2] + prob_a1_c_y0_u[1] * inverse[1][2] + prob_a1_c_y0_u[2] * \
             inverse[2][2] + prob_a1_c_y0_u[3] * inverse[3][2]
    denom_2a = num_2a + tmp_2a
    upper_2a = num_2a / denom_2a
    
    # Getting P(Y=1 | A=0, C, U=2)
    num_2b = prob_a0_c_y1_u[0] * inverse[0][2] + prob_a0_c_y1_u[1] * inverse[1][2] + prob_a0_c_y1_u[2] * \
             inverse[2][2] + prob_a0_c_y1_u[3] * inverse[3][2]
    tmp_2b = prob_a0_c_y0_u[0] * inverse[0][2] + prob_a0_c_y0_u[1] * inverse[1][2] + prob_a0_c_y0_u[2] * \
             inverse[2][2] + prob_a0_c_y0_u[3] * inverse[3][2]
    denom_2b = num_2b + tmp_2b
    lower_2b = num_2b / denom_2b
    
    # comp_2 = upper_2a / lower_2b
    
    # Getting P(Y=1 | A=1, C, U=3)
    num_3a = prob_a1_c_y1_u[0] * inverse[0][3] + prob_a1_c_y1_u[1] * inverse[1][3] + prob_a1_c_y1_u[2] * \
             inverse[2][3] + prob_a1_c_y1_u[3] * inverse[3][3]
    tmp_3a = prob_a1_c_y0_u[0] * inverse[0][3] + prob_a1_c_y0_u[1] * inverse[1][3] + prob_a1_c_y0_u[2] * \
             inverse[2][3] + prob_a1_c_y0_u[3] * inverse[3][3]
    denom_3a = num_3a + tmp_3a
    upper_3a = num_3a / denom_3a
    
    # Getting P(Y=1 | A=0, C, U=3)
    num_3b = prob_a0_c_y1_u[0] * inverse[0][3] + prob_a0_c_y1_u[1] * inverse[1][3] + prob_a0_c_y1_u[2] * \
             inverse[2][3] + prob_a0_c_y1_u[3] * inverse[3][3]
    tmp_3b = prob_a0_c_y0_u[0] * inverse[0][3] + prob_a0_c_y0_u[1] * inverse[1][3] + prob_a0_c_y0_u[2] * \
             inverse[2][3] + prob_a0_c_y0_u[3] * inverse[3][3]
    denom_3b = num_3b + tmp_3b
    lower_3b = num_3b / denom_3b
    
    # comp_3 = upper_3a / lower_3b
    
    # Getting P(u | c) 
    prob_u0_c = num_0a + tmp_0a + num_0b + tmp_0b
    prob_u1_c = num_1a + tmp_1a + num_1b + tmp_1b
    prob_u2_c = num_2a + tmp_2a + num_2b + tmp_2b
    prob_u3_c = num_3a + tmp_3a + num_3b + tmp_3b
    
    total_upper = np.mean(upper_0a * prob_u0_c + upper_1a * prob_u1_c + upper_2a * prob_u2_c + \
                          upper_3a * prob_u3_c)
    total_lower = np.mean(lower_0b * prob_u0_c + lower_1b * prob_u1_c + lower_2b * prob_u2_c + \
                          lower_3b * prob_u3_c)
    
    
    
    rr = total_upper / total_lower
    sub_array = [np.mean(upper_0a * prob_u0_c) / np.mean(lower_0b * prob_u0_c),
                 np.mean(upper_1a * prob_u1_c) / np.mean(lower_1b * prob_u1_c),
                 np.mean(upper_2a * prob_u2_c) / np.mean(lower_2b * prob_u2_c),
                 np.mean(upper_3a * prob_u3_c) / np.mean(lower_3b * prob_u3_c)]
    return rr, sub_array

In [491]:
def bootstrap(dataframe, model1, model2, model3):
    '''
    Given a dataframe and 3 models generated from generate_models(), bootstrap the testing set 
    for n2c2 2006 smoking dataset to get different error rate matrices to test robustness of the 
    risk ratio casual effect. Iterations set to 100
    Utilize predict_bootstrap_2006.py to generate pickle files that store the confusion matrices.
    '''
    
    # Iterating through the bootstrapped confusion matrices 
    # "iterations" var depends on how many bootstrapped confusion matrics were generated
    iterations = 100
    rr_arr = []
    sub_matrix = np.zeros((iterations, 4))
    cnt = 0
    for x in range(0, iterations):
        f = open("..." + str(x) + ".pkl", "rb")
        con_matrix = pickle.load(f)
        if con_matrix.shape[0] > 4:
            con_matrix = con_matrix[1:, 1:]
        res = con_matrix/con_matrix.sum(axis=1)[:,None]
        
        rr, sub_array = risk_ratio_bootstrap(dataframe, model1, model2, model3, res)
        rr_arr.append(rr)
        sub_avg = sub_matrix.mean(axis=0)

    res_dict = {"bt_rr": sum(rr_arr) / len(rr_arr), "bt_arr_rr": rr_arr, "sub_avg_rr": sub_avg, \
                "sub_arr_rr": sub_matrix}
    
    return res_dict

In [ ]:
m1, m2, m3 = generate_models(merged_df)
bt_rr = bootstrap(merged_df, m1, m2, m3)
bt_rr["bt_rr"]

In [441]:
# Computing 95% interval for risk ratio while boostrapping error rate matrix
compute_ci_95(bt_rr["bt_arr_rr"])

[0.8963576055234153, 0.965890589280091]

In [442]:
compute_ci_95(bt_rr["sub_arr_rr"][:, 2])

[0.8989310050963705, 0.95523635359726]

In [443]:
# Computing 95% interval for risk ratio subgroups while
# bootstrapping error rate matrix
np.apply_along_axis(compute_ci_95, axis=0, arr=bt_rr["sub_arr_rr"])

array([[ 0.88170568,  0.89532019,  0.89893101, -4.8111567 ],
       [ 0.97747752,  0.96495761,  0.95523635,  4.77866405]])

### Combined bootstrapping for RR

In [444]:
def combined_bootstrap_rr(dataframe, model1, model2, model3):
    '''
    Given a dataframe and 3 models generated from generate_models(), do combined bootstrapping to see
    robustness of risk ratio calculations. Iterations set to 100
    Utilize predict_bootstrap_2006.py to generate pickle files that store the confusion matrices.
    '''
    
    # Iterate through 10 of bootstrapped error matrices
    iterations_em = 10
    rr_arr = []
    for iem in range(iterations_em):
        f = open("..." + str(iem) + ".pkl", "rb")
        con_matrix = pickle.load(f)
        if con_matrix.shape[0] > 4:
            con_matrix = con_matrix[1:, 1:]
        res = con_matrix/con_matrix.sum(axis=1)[:,None]
        
        # Iterate through 10 bootstrapped (shuffled) dataframes
        iterations_df = 10
        for idf in range(iterations_em):
            bt_df = dataframe.sample(frac=1, replace=True, ignore_index=True)
            rr, sub_array = risk_ratio_bootstrap(bt_df, model1, model2, model3, res)
            rr_arr.append(rr)
    print("Number of calcs:", len(rr_arr)) # == 100 based on default settings
    print("Mean combined bootstrap rr:", sum(rr_arr) / len(rr_arr))
    return [sum(rr_arr) / len(rr_arr), rr_arr]
        

In [445]:
### Compute 95% invertal for RR while doing combined bootstrapping
combined_rr = combined_bootstrap_rr(merged_df, m1, m2, m3)
compute_ci_95(combined_rr[1])

Number of calcs: 100
Mean combined bootstrap rr: 0.9307894835004851


[0.9115170643319347, 0.9960319931234776]

# Implementing OR

In [546]:
def odds_ratio(dataframe, model1, model2, model3):
    '''
    Given a pre-procesesed MIMIC + smoking proxy prediction dataframe as well as three trained models 
    from generate_models(), calculate the odds ratio as defined by: 
    causal_effect = (P(Y^{a=1}=1) * P(Y^{a=0}=0)) / (P(Y^{a=1}=0) * P(Y^{a=0}=1))
    The assumptions of this function are:
        1) Smoking proxy predictions are categorical
        2) Treatment variable values are binary --> either 1 for receiving treatment or 0 
           for not receiving treatment
        3) Order for model inputs matter:
            a) model1 = P(y | u*, a, c) --> y ~ u* + a + c
            b) model2 = P(u* | a, c)
            c) model3 = P(a | c)
        4) Default prediction is probability of getting 1 due to how statsmodels works
    '''
    
    tmp_df = None
    unique_smoking = [1,2,3,4]
    unique_echo = [1,0]
    exp_array = []
    
    # Understanding Matrix of Error Adjustments
    confusion = [
                [8, 0, 2, 1],
                [4, 4, 3, 0],
                [1, 0, 14, 1],
                [1, 0, 1, 61]
            ] # rows represent the ground truth labels and cols represents the predicted labels

    error_mat = [
                    [8/11, 0, 2/11, 1/11],
                    [4/11, 4/11, 3/11, 0],
                    [1/16, 0, 14/16, 1/16],
                    [1/63, 0, 1/63, 61/63]
            ]
#     confusion = [
#                 [167, 5, 17, 0],
#                 [20, 70, 22, 1],
#                 [13, 10, 330, 3],
#                 [8, 0, 12, 11]
#             ] # rows represent the ground truth labels and cols represents the predicted labels

#     error_mat = [
#                     [167/189, 5/189, 17/189, 0/189],
#                     [20/113, 70/113, 22/113, 1/113],
#                     [13/356, 10/356, 330/356, 3/356],
#                     [8/31, 0, 12/31, 11/31]
#                 ] # rows represent U* and cols represent U
    inverse = np.linalg.pinv(error_mat)
    
    # Getting P(A, c, y=1, u*) 
    prob_a1_c_y1_u = []
    prob_a0_c_y1_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to be a cateogrical value in [1,2,3,4]
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y1_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y1_u.append(output)
    
    # Getting P(A, c, y=0, u*)
    prob_a1_c_y0_u = []
    prob_a0_c_y0_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to either be 1 or 0
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = 1 - model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y0_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y0_u.append(output)
    
    # Getting P(Y=1 | A=1, C, U=0)
    num_0a = prob_a1_c_y1_u[0] * inverse[0][0] + prob_a1_c_y1_u[1] * inverse[1][0] + prob_a1_c_y1_u[2] * \
             inverse[2][0] + prob_a1_c_y1_u[3] * inverse[3][0]
    tmp_0a = prob_a1_c_y0_u[0] * inverse[0][0] + prob_a1_c_y0_u[1] * inverse[1][0] + prob_a1_c_y0_u[2] * \
             inverse[2][0] + prob_a1_c_y0_u[3] * inverse[3][0]
    denom_0a = num_0a + tmp_0a
    upper_0a = num_0a / denom_0a
    
    # Getting P(Y=1 | A=0, C, U=0)
    num_0b = prob_a0_c_y1_u[0] * inverse[0][0] + prob_a0_c_y1_u[1] * inverse[1][0] + prob_a0_c_y1_u[2] * \
             inverse[2][0] + prob_a0_c_y1_u[3] * inverse[3][0]
    tmp_0b = prob_a0_c_y0_u[0] * inverse[0][0] + prob_a0_c_y0_u[1] * inverse[1][0] + prob_a0_c_y0_u[2] * \
             inverse[2][0] + prob_a0_c_y0_u[3] * inverse[3][0]
    denom_0b = num_0b + tmp_0b
    lower_0b = num_0b / denom_0b
    
    
    # Getting P(Y=1 | A=1, C, U=1)
    num_1a = prob_a1_c_y1_u[0] * inverse[0][1] + prob_a1_c_y1_u[1] * inverse[1][1] + prob_a1_c_y1_u[2] * \
             inverse[2][1] + prob_a1_c_y1_u[3] * inverse[3][1]
    tmp_1a = prob_a1_c_y0_u[0] * inverse[0][1] + prob_a1_c_y0_u[1] * inverse[1][1] + prob_a1_c_y0_u[2] * \
             inverse[2][1] + prob_a1_c_y0_u[3] * inverse[3][1]
    denom_1a = num_1a + tmp_1a
    upper_1a = num_1a / denom_1a
    
    # Getting P(Y=1 | A=0, C, U=1)
    num_1b = prob_a0_c_y1_u[0] * inverse[0][1] + prob_a0_c_y1_u[1] * inverse[1][1] + prob_a0_c_y1_u[2] * \
             inverse[2][1] + prob_a0_c_y1_u[3] * inverse[3][1]
    tmp_1b = prob_a0_c_y0_u[0] * inverse[0][1] + prob_a0_c_y0_u[1] * inverse[1][1] + prob_a0_c_y0_u[2] * \
             inverse[2][1] + prob_a0_c_y0_u[3] * inverse[3][1]
    denom_1b = num_1b + tmp_1b
    lower_1b = num_1b / denom_1b
    
    
    # Getting P(Y=1 | A=1, C, U=2)
    num_2a = prob_a1_c_y1_u[0] * inverse[0][2] + prob_a1_c_y1_u[1] * inverse[1][2] + prob_a1_c_y1_u[2] * \
             inverse[2][2] + prob_a1_c_y1_u[3] * inverse[3][2]
    tmp_2a = prob_a1_c_y0_u[0] * inverse[0][2] + prob_a1_c_y0_u[1] * inverse[1][2] + prob_a1_c_y0_u[2] * \
             inverse[2][2] + prob_a1_c_y0_u[3] * inverse[3][2]
    denom_2a = num_2a + tmp_2a
    upper_2a = num_2a / denom_2a
    
    # Getting P(Y=1 | A=0, C, U=2)
    num_2b = prob_a0_c_y1_u[0] * inverse[0][2] + prob_a0_c_y1_u[1] * inverse[1][2] + prob_a0_c_y1_u[2] * \
             inverse[2][2] + prob_a0_c_y1_u[3] * inverse[3][2]
    tmp_2b = prob_a0_c_y0_u[0] * inverse[0][2] + prob_a0_c_y0_u[1] * inverse[1][2] + prob_a0_c_y0_u[2] * \
             inverse[2][2] + prob_a0_c_y0_u[3] * inverse[3][2]
    denom_2b = num_2b + tmp_2b
    lower_2b = num_2b / denom_2b
    
    # Getting P(Y=1 | A=1, C, U=3)
    num_3a = prob_a1_c_y1_u[0] * inverse[0][3] + prob_a1_c_y1_u[1] * inverse[1][3] + prob_a1_c_y1_u[2] * \
             inverse[2][3] + prob_a1_c_y1_u[3] * inverse[3][3]
    tmp_3a = prob_a1_c_y0_u[0] * inverse[0][3] + prob_a1_c_y0_u[1] * inverse[1][3] + prob_a1_c_y0_u[2] * \
             inverse[2][3] + prob_a1_c_y0_u[3] * inverse[3][3]
    denom_3a = num_3a + tmp_3a
    upper_3a = num_3a / denom_3a
    
    # Getting P(Y=1 | A=0, C, U=3)
    num_3b = prob_a0_c_y1_u[0] * inverse[0][3] + prob_a0_c_y1_u[1] * inverse[1][3] + prob_a0_c_y1_u[2] * \
             inverse[2][3] + prob_a0_c_y1_u[3] * inverse[3][3]
    tmp_3b = prob_a0_c_y0_u[0] * inverse[0][3] + prob_a0_c_y0_u[1] * inverse[1][3] + prob_a0_c_y0_u[2] * \
             inverse[2][3] + prob_a0_c_y0_u[3] * inverse[3][3]
    denom_3b = num_3b + tmp_3b
    lower_3b = num_3b / denom_3b
    
    # Getting P(u | c) 
    prob_u0_c = num_0a + tmp_0a + num_0b + tmp_0b
    prob_u1_c = num_1a + tmp_1a + num_1b + tmp_1b
    prob_u2_c = num_2a + tmp_2a + num_2b + tmp_2b
    prob_u3_c = num_3a + tmp_3a + num_3b + tmp_3b
    
    numerator_a = np.sum(upper_0a * prob_u0_c + upper_1a * prob_u1_c + upper_2a * prob_u2_c + \
                         upper_3a * prob_u3_c)
    numerator_b = np.sum((1 - lower_0b) * prob_u0_c + (1 - lower_1b) * prob_u1_c + \
                         (1 - lower_2b) * prob_u2_c + (1 - lower_3b) * prob_u3_c)
    
    
    denominator_a = np.sum((1 - upper_0a) * prob_u0_c + (1 - upper_1a) * prob_u1_c + \
                           (1 - upper_2a) * prob_u2_c + (1 - upper_3a) * prob_u3_c)
    denominator_b = np.sum(lower_0b * prob_u0_c + lower_1b * prob_u1_c + lower_2b * prob_u2_c + \
                           lower_3b * prob_u3_c)
    
    numerator = numerator_a * numerator_b
    denominator = denominator_a * denominator_b
    
    return numerator / denominator
    

In [547]:
m1, m2, m3 = generate_models(merged_df)
odds_ratio(merged_df, m1, m2, m3)

0.8864763609831658

### Bootstrapping merged dataframe for OR

In [515]:
def bootstrap_merged_data_or(dataframe, m1, m2, m3):
    '''
    Given a pre-procesesed MIMIC + smoking proxy prediction dataframe as well as three trained models 
    from generate_models(), perform bootstrapping by shuffling the merged dataframe for
    odds ratio calculations. Iterations set to 100.
    The assumptions of this function are:
        1) Smoking proxy predictions are categorical
        2) Treatment variable values are binary --> either 1 for receiving treatment or 0 
           for not receiving treatment
        3) Order for model inputs matter:
            a) model1 = P(y | u*, a, c) --> y ~ u* + a + c
            b) model2 = P(u* | a, c)
            c) model3 = P(a | c)
    '''
    iterations = 100
    output = []
    
    for _ in range(iterations):
        bt_data = dataframe.sample(frac=1, replace=True, ignore_index=True)
        or_val = odds_ratio(bt_data, m1, m2, m3)
        output.append(or_val)

    return [sum(output) / len(output), output]

bt_merged_or = bootstrap_merged_data_or(merged_df, m1, m2, m3)
bt_merged_or[0]

0.9417611891942418

In [516]:
# Compute 95% CI for OR while bootstrapping merged dataframe
compute_ci_95(bt_merged_or[1])

[0.8973625487342399, 1.0263516361373635]

### Bootstrapping Error Matrix for OR

In [517]:
def odds_ratio_bootstrap(dataframe, model1, model2, model3, error_mat):
    '''
    Given a pre-procesesed MIMIC + smoking proxy prediction dataframe, three trained models 
    from generate_models(), and an error rate matrix, perform bootstrapping on error-rate matrix for
    odds ratio calculations. Helper function for bootstrap_or()
    The assumptions of this function are:
        1) Smoking proxy predictions are categorical
        2) Treatment variable values are binary --> either 1 for receiving treatment or 0 
           for not receiving treatment
        3) Order for model inputs matter:
            a) model1 = P(y | u*, a, c) --> y ~ u* + a + c
            b) model2 = P(u* | a, c)
            c) model3 = P(a | c)
    '''
    tmp_df = None
    unique_smoking = [1,2,3,4]
    unique_echo = [1,0]
    exp_array = []
    
    inverse = np.linalg.pinv(error_mat)
    
    # Getting P(A, c, y=1, u*) 
    prob_a1_c_y1_u = []
    prob_a0_c_y1_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to be a cateogrical value in [1,2,3,4]
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y1_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y1_u.append(output)
    
    # Getting P(A, c, y=0, u*)
    prob_a1_c_y0_u = []
    prob_a0_c_y0_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to either be 1 or 0
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = 1 - model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y0_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y0_u.append(output)
    
    # Getting P(Y=1 | A=1, C, U=0)
    num_0a = prob_a1_c_y1_u[0] * inverse[0][0] + prob_a1_c_y1_u[1] * inverse[1][0] + prob_a1_c_y1_u[2] * \
             inverse[2][0] + prob_a1_c_y1_u[3] * inverse[3][0]
    tmp_0a = prob_a1_c_y0_u[0] * inverse[0][0] + prob_a1_c_y0_u[1] * inverse[1][0] + prob_a1_c_y0_u[2] * \
             inverse[2][0] + prob_a1_c_y0_u[3] * inverse[3][0]
    denom_0a = num_0a + tmp_0a
    upper_0a = num_0a / denom_0a
    
    # Getting P(Y=1 | A=0, C, U=0)
    num_0b = prob_a0_c_y1_u[0] * inverse[0][0] + prob_a0_c_y1_u[1] * inverse[1][0] + prob_a0_c_y1_u[2] * \
             inverse[2][0] + prob_a0_c_y1_u[3] * inverse[3][0]
    tmp_0b = prob_a0_c_y0_u[0] * inverse[0][0] + prob_a0_c_y0_u[1] * inverse[1][0] + prob_a0_c_y0_u[2] * \
             inverse[2][0] + prob_a0_c_y0_u[3] * inverse[3][0]
    denom_0b = num_0b + tmp_0b
    lower_0b = num_0b / denom_0b
    
    
    # Getting P(Y=1 | A=1, C, U=1)
    num_1a = prob_a1_c_y1_u[0] * inverse[0][1] + prob_a1_c_y1_u[1] * inverse[1][1] + prob_a1_c_y1_u[2] * \
             inverse[2][1] + prob_a1_c_y1_u[3] * inverse[3][1]
    tmp_1a = prob_a1_c_y0_u[0] * inverse[0][1] + prob_a1_c_y0_u[1] * inverse[1][1] + prob_a1_c_y0_u[2] * \
             inverse[2][1] + prob_a1_c_y0_u[3] * inverse[3][1]
    denom_1a = num_1a + tmp_1a
    upper_1a = num_1a / denom_1a
    
    # Getting P(Y=1 | A=0, C, U=1)
    num_1b = prob_a0_c_y1_u[0] * inverse[0][1] + prob_a0_c_y1_u[1] * inverse[1][1] + prob_a0_c_y1_u[2] * \
             inverse[2][1] + prob_a0_c_y1_u[3] * inverse[3][1]
    tmp_1b = prob_a0_c_y0_u[0] * inverse[0][1] + prob_a0_c_y0_u[1] * inverse[1][1] + prob_a0_c_y0_u[2] * \
             inverse[2][1] + prob_a0_c_y0_u[3] * inverse[3][1]
    denom_1b = num_1b + tmp_1b
    lower_1b = num_1b / denom_1b
    
    
    # Getting P(Y=1 | A=1, C, U=2)
    num_2a = prob_a1_c_y1_u[0] * inverse[0][2] + prob_a1_c_y1_u[1] * inverse[1][2] + prob_a1_c_y1_u[2] * \
             inverse[2][2] + prob_a1_c_y1_u[3] * inverse[3][2]
    tmp_2a = prob_a1_c_y0_u[0] * inverse[0][2] + prob_a1_c_y0_u[1] * inverse[1][2] + prob_a1_c_y0_u[2] * \
             inverse[2][2] + prob_a1_c_y0_u[3] * inverse[3][2]
    denom_2a = num_2a + tmp_2a
    upper_2a = num_2a / denom_2a
    
    # Getting P(Y=1 | A=0, C, U=2)
    num_2b = prob_a0_c_y1_u[0] * inverse[0][2] + prob_a0_c_y1_u[1] * inverse[1][2] + prob_a0_c_y1_u[2] * \
             inverse[2][2] + prob_a0_c_y1_u[3] * inverse[3][2]
    tmp_2b = prob_a0_c_y0_u[0] * inverse[0][2] + prob_a0_c_y0_u[1] * inverse[1][2] + prob_a0_c_y0_u[2] * \
             inverse[2][2] + prob_a0_c_y0_u[3] * inverse[3][2]
    denom_2b = num_2b + tmp_2b
    lower_2b = num_2b / denom_2b
    
    # Getting P(Y=1 | A=1, C, U=3)
    num_3a = prob_a1_c_y1_u[0] * inverse[0][3] + prob_a1_c_y1_u[1] * inverse[1][3] + prob_a1_c_y1_u[2] * \
             inverse[2][3] + prob_a1_c_y1_u[3] * inverse[3][3]
    tmp_3a = prob_a1_c_y0_u[0] * inverse[0][3] + prob_a1_c_y0_u[1] * inverse[1][3] + prob_a1_c_y0_u[2] * \
             inverse[2][3] + prob_a1_c_y0_u[3] * inverse[3][3]
    denom_3a = num_3a + tmp_3a
    upper_3a = num_3a / denom_3a
    
    # Getting P(Y=1 | A=0, C, U=3)
    num_3b = prob_a0_c_y1_u[0] * inverse[0][3] + prob_a0_c_y1_u[1] * inverse[1][3] + prob_a0_c_y1_u[2] * \
             inverse[2][3] + prob_a0_c_y1_u[3] * inverse[3][3]
    tmp_3b = prob_a0_c_y0_u[0] * inverse[0][3] + prob_a0_c_y0_u[1] * inverse[1][3] + prob_a0_c_y0_u[2] * \
             inverse[2][3] + prob_a0_c_y0_u[3] * inverse[3][3]
    denom_3b = num_3b + tmp_3b
    lower_3b = num_3b / denom_3b
    
    # Getting P(u | c) 
    prob_u0_c = num_0a + tmp_0a + num_0b + tmp_0b
    prob_u1_c = num_1a + tmp_1a + num_1b + tmp_1b
    prob_u2_c = num_2a + tmp_2a + num_2b + tmp_2b
    prob_u3_c = num_3a + tmp_3a + num_3b + tmp_3b
     
    numerator_a = np.sum(upper_0a * prob_u0_c + upper_1a * prob_u1_c + upper_2a * prob_u2_c + \
                         upper_3a * prob_u3_c)
    numerator_b = np.sum((1 - lower_0b) * prob_u0_c + (1 - lower_1b) * prob_u1_c + \
                         (1 - lower_2b) * prob_u2_c + (1 - lower_3b) * prob_u3_c)
    
    denominator_a = np.sum((1 - upper_0a) * prob_u0_c + (1 - upper_1a) * prob_u1_c + \
                           (1 - upper_2a) * prob_u2_c + (1 - upper_3a) * prob_u3_c)
    denominator_b = np.sum(lower_0b * prob_u0_c + lower_1b * prob_u1_c + lower_2b * prob_u2_c + \
                           lower_3b * prob_u3_c)
    
    numerator = numerator_a * numerator_b
    denominator = denominator_a * denominator_b
    
    return numerator / denominator

In [518]:
def bootstrap_or(dataframe, model1, model2, model3):
    '''
    Given a dataframe and 3 models generated from generate_models(), bootstrap the testing set 
    for n2c2 2006 smoking dataset to get different error rate matrices to test robustness of 
    the odds ratio casual effect.
    Utilize predict_bootstrap_2006.py to generate pickle files that store the confusion matrices.
    '''
    
    # Iterating through the bootstrapped confusion matrices 
    # "iterations" var depends on how many bootstrapped confusion matrics were generated
    # Default in predict_bootstrap_2006.py is 10
    iterations = 100
    o_r_arr = []
    for x in range(iterations):
        f = open("..." + str(x) + ".pkl", "rb")
        con_matrix = pickle.load(f)
        print(x, con_matrix.shape)
        if con_matrix.shape[0] > 4:
            con_matrix = con_matrix[1:, 1:]
        res = con_matrix/con_matrix.sum(axis=1)[:,None]
        o_r = odds_ratio_bootstrap(dataframe, model1, model2, model3, res)
        o_r_arr.append(o_r)
    
    return [sum(o_r_arr) / len(o_r_arr), o_r_arr]

In [ ]:
m1, m2, m3 = generate_models(merged_df)
bt_or = bootstrap_or(merged_df, m1, m2, m3)
bt_or[0]

In [520]:
# Computing 95% interval for odds ratio while boostrapping error rate matrix
compute_ci_95(bt_or[1])

[0.6443861637961674, 1.1151837856585642]

### Combined Bootstrapping for OR

In [521]:
def combined_bootstrap_or(dataframe, model1, model2, model3):
    '''
    Given a dataframe and 3 models generated from generate_models(), do combined bootstrapping to see
    robustness of odds ratio calculations. Iterations set to 100
    Utilize predict_bootstrap_2006.py to generate pickle files that store the confusion matrices.
    '''
    
    # Iterate through 10 of bootstrapped error matrices
    iterations_em = 10
    or_arr = []
    for iem in range(iterations_em):
        f = open("..." + str(iem) + ".pkl", "rb")
        con_matrix = pickle.load(f)
        if con_matrix.shape[0] > 4:
            con_matrix = con_matrix[1:, 1:]
        res = con_matrix/con_matrix.sum(axis=1)[:,None]
        
        # Iterate through 10 bootstrapped (shuffled) dataframes
        iterations_df = 10
        for idf in range(iterations_df):
            bt_df = dataframe.sample(frac=1, replace=True, ignore_index=True)
            or_v = odds_ratio_bootstrap(bt_df, model1, model2, model3, res)
            or_arr.append(or_v)
    print(len(or_arr)) # == 100 based on default settings
    print("Mean combined bootstrap or_v:", sum(or_arr) / len(or_arr))
    return [sum(or_arr) / len(or_arr), or_arr]

In [522]:
### Compute 95% invertal for OR while doing combined bootstrapping
combined_or = combined_bootstrap_or(merged_df, m1, m2, m3)
compute_ci_95(combined_or[1])

100
Mean combined bootstrap or_v: 0.8689663893395795


[0.48126824268046753, 1.001330736240231]